**Automated Response Generation for Customer Support**

In [ ]:
!pip install datasets transformers rouge_score

In [ ]:
!pip install transformers --upgrade

In [ ]:
from datasets import load_dataset
from transformers import BartTokenizer, BartForConditionalGeneration, Trainer, TrainingArguments
from transformers import DataCollatorForSeq2Seq

# Step 1: Load the dataset
dataset = load_dataset("Kaludi/Customer-Support-Responses")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['query', 'response'],
        num_rows: 74
    })
})

In [ ]:
dataset = dataset.rename_column("query", "question")
dataset = dataset.rename_column("response", "answer")


In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 74
    })
})

In [ ]:
# Step 2: Initialize the tokenizer and model
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large')

In [ ]:
# Step 3: Preprocess the dataset
def preprocess_function(examples):
    inputs = examples["question"]
    targets = examples["answer"]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=1024, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = dataset.map(preprocess_function, batched=True)

In [ ]:
# Step 4: Initialize the data collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)


In [ ]:
!pip install transformers[torch]

In [ ]:
!pip install accelerate -U

In [ ]:
!pip install transformers -U

In [ ]:
# Step 5: Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    #predict_with_generate=True,
    fp16=True,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 74
    })
})

In [ ]:
# Splitting the dataset into training and testing dataset
train_test_split = tokenized_datasets["train"].train_test_split(test_size=0.3, shuffle=False, seed=42)

tokenized_datasets["train"] = train_test_split["train"]
tokenized_datasets["test"] = train_test_split["test"]


In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 51
    })
    test: Dataset({
        features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 23
    })
})

In [ ]:
# Step 6: Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [ ]:
# Step 7: Train the model
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,1.384238
2,No log,1.099502
3,No log,1.020432


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


TrainOutput(global_step=39, training_loss=1.843948755508814, metrics={'train_runtime': 547.0314, 'train_samples_per_second': 0.28, 'train_steps_per_second': 0.071, 'total_flos': 3991366483968.0, 'train_loss': 1.843948755508814, 'epoch': 3.0})

In [ ]:
# Step 8: Save the model
model.save_pretrained("./finetuned_bart")
tokenizer.save_pretrained("./finetuned_bart")


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


('./finetuned_bart/tokenizer_config.json',
 './finetuned_bart/special_tokens_map.json',
 './finetuned_bart/vocab.json',
 './finetuned_bart/merges.txt',
 './finetuned_bart/added_tokens.json')

In [ ]:
!pip install rouge_score

In [ ]:
# Step 9: Evaluate the model
from datasets import load_metric
metric = load_metric("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = [[label] for label in labels]
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    return {k: round(v, 4) for k, v in result.items()}

results = trainer.evaluate()
print(results)

<ipython-input-22-874b8e61c8c7>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("rouge")


The repository for rouge contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/rouge.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


{'eval_loss': 1.0204322338104248, 'eval_runtime': 23.9455, 'eval_samples_per_second': 0.961, 'eval_steps_per_second': 0.251, 'epoch': 3.0}


In [ ]:
# Step 10: Implementing a demo in Jupyter notebook
import ipywidgets as widgets
from IPython.display import display

# Function to generate a response
def generate_response(question):
    inputs = tokenizer(question, return_tensors="pt", max_length=1024, truncation=True)
    outputs = model.generate(inputs.input_ids, max_length=1024, num_beams=4, early_stopping=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Create widgets
input_box = widgets.Textarea(placeholder='Type your question here...', description='Question:')
output_box = widgets.Textarea(placeholder='Generated response will appear here...', description='Response:', disabled=True)
button = widgets.Button(description='Generate Response')

# Define the button click event
def on_button_clicked(b):
    question = input_box.value
    response = generate_response(question)
    output_box.value = response

button.on_click(on_button_clicked)

# Display the widgets
display(input_box, button, output_box)

Textarea(value='', description='Question:', placeholder='Type your question here...')

Button(description='Generate Response', style=ButtonStyle())

Textarea(value='', description='Response:', disabled=True, placeholder='Generated response will appear here...…

**Demo**

In [ ]:
# Import necessary libraries
import torch
from transformers import BartTokenizer, BartForConditionalGeneration
import ipywidgets as widgets
from IPython.display import display, HTML

# Load the fine-tuned model and tokenizer
model_path = "./finetuned_bart"
model = BartForConditionalGeneration.from_pretrained(model_path)
tokenizer = BartTokenizer.from_pretrained(model_path)

# Define the function to generate responses
def generate_response(question):
    inputs = tokenizer(question, return_tensors="pt", max_length=1024, truncation=True)
    outputs = model.generate(inputs.input_ids, max_length=1024, num_beams=4, early_stopping=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [28]:
# Create interactive widgets
input_box = widgets.Textarea(
    placeholder='Type your question here...',
    description='Question:',
    layout=widgets.Layout(width='80%', height='100px')
)
output_box = widgets.Textarea(
    placeholder='Generated response will appear here...',
    description='Response:',
    layout=widgets.Layout(width='80%', height='100px'),
    disabled=True
)
button = widgets.Button(description='Generate Response', button_style='success')
sample_dropdown = widgets.Dropdown(
    options=[
        "How can I reset my password?",
        "What is the refund policy for my order?",
        "How do I track my shipment?",
        "Can I change my delivery address after placing an order?",
        "What are the accepted payment methods?"
    ],
    description='Sample Queries:',
    layout=widgets.Layout(width='80%')
)

# Define the button click event
def on_button_clicked(b):
    question = input_box.value
    response = generate_response(question)
    output_box.value = response

button.on_click(on_button_clicked)

# Define the sample query selection event
def on_sample_selected(change):
    input_box.value = sample_dropdown.value

sample_dropdown.observe(on_sample_selected, names='value')

# Display the interface
display(HTML("<h2>Customer Support Response Generator</h2>"))
display(HTML("<p>Type a question or select a sample query from the dropdown below and click 'Generate Response' to see the model's answer.</p>"))
display(sample_dropdown)
display(input_box)
display(button)
display(output_box)

# Provide a function to test sample queries
def test_sample_queries():
    for query in sample_dropdown.options:
        print(f"Question: {query}")
        response = generate_response(query)
        print(f"Response: {response}\n")

# Uncomment the following line to test the sample queries directly
# test_sample_queries()

Dropdown(description='Sample Queries:', layout=Layout(width='80%'), options=('How can I reset my password?', '…

Textarea(value='', description='Question:', layout=Layout(height='100px', width='80%'), placeholder='Type your…

Button(button_style='success', description='Generate Response', style=ButtonStyle())

Textarea(value='', description='Response:', disabled=True, layout=Layout(height='100px', width='80%'), placeho…

In [29]:
test_sample_queries()

Question: How can I reset my password?
Response: We apologize for the inconvenience. Can you please provide your email so we can reset your password?

Question: What is the refund policy for my order?
Response: We apologize for the inconvenience. Can you please provide your order number so we can provide you with a refund policy?

Question: How do I track my shipment?
Response: We'd be happy to help. Can you please provide your order number and the product name or SKU you're interested in?

Question: Can I change my delivery address after placing an order?
Response: We'd be happy to help. Can you please provide your order number and the address you'd like to change?

Question: What are the accepted payment methods?
Response: We'd be happy to help. Can you please provide your account email so we can provide you with the accepted payment method?

